In [9]:
import os
import warnings
import joblib
import numpy as np
import pandas as pd
import optuna

from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    ExtraTreesRegressor, AdaBoostRegressor
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, LinearRegression

# Basic setup
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Load dataset
file_path = os.path.join(os.getcwd(), "HEAs-ML_training.xlsx")
data = pd.read_excel(file_path)

# Features and target (last column = YS)
X_df = data.iloc[:, :-1].copy()
y_ser = data.iloc[:, -1].copy()

# Ensure numeric and convert to numpy arrays
X = X_df.apply(pd.to_numeric, errors="raise").to_numpy()
y = pd.to_numeric(y_ser, errors="raise").to_numpy()

# 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

# Model definitions and search spaces (only this part tuned)
model_dict = {
    "XGB": (XGBRegressor, {
        "n_estimators": (100, 600),
        "max_depth": (3, 12),
        "learning_rate": (0.005, 0.3),
        "colsample_bytree": (0.6, 1.0),
        "subsample": (0.6, 1.0)
    }),
    "RF": (RandomForestRegressor, {
        "n_estimators": (100, 600),
        "max_depth": (3, 20),
        "min_samples_split": (4, 40),
        "min_samples_leaf": (2, 15)
    }),
    "GBR": (GradientBoostingRegressor, {
        "n_estimators": (100, 600),
        "max_depth": (2, 8),
        "learning_rate": (0.005, 0.2),
        "subsample": (0.6, 1.0)
    }),
    "LGB": (LGBMRegressor, {
        "n_estimators": (100, 600),
        "max_depth": (3, 15),
        "learning_rate": (0.005, 0.2),
        "subsample": (0.6, 1.0),
        "colsample_bytree": (0.6, 1.0)
    }),
    "ADA": (AdaBoostRegressor, {
        "n_estimators": (50, 400),
        "learning_rate": (0.01, 0.5)
    }),
    "DT": (DecisionTreeRegressor, {
        "max_depth": (2, 15),
        "min_samples_split": (4, 40),
        "min_samples_leaf": (3, 20)
    }),
    "ET": (ExtraTreesRegressor, {
        "n_estimators": (100, 600),
        "max_depth": (3, 20),
        "min_samples_split": (4, 40),
        "min_samples_leaf": (2, 15)
    }),
    "Ridge": (Ridge, {"alpha": (1e-3, 100.0)}),
}

os.makedirs("OptimizationLogs", exist_ok=True)
results = []


def build_model(model_class, model_name, params):
    base = model_class()
    kwargs = params.copy()
    if "random_state" in base.get_params():
        kwargs["random_state"] = RANDOM_STATE
    if "n_jobs" in base.get_params():
        kwargs["n_jobs"] = -1
    if model_name == "XGB":
        kwargs["tree_method"] = "hist"
        kwargs["device"] = "cuda"
    if model_name == "LGB":
        kwargs["verbose"] = -1
    return model_class(**kwargs)


# Hyperparameter tuning with 5-fold CV
for model_name, (model_class, param_dict) in model_dict.items():

    def objective(trial):
        params = {}
        for key, val in param_dict.items():
            if isinstance(val[0], int):
                params[key] = trial.suggest_int(key, val[0], val[1])
            else:
                params[key] = trial.suggest_float(key, val[0], val[1], log=True)

        model = build_model(model_class, model_name, params)
        mape_list = []

        for tr_idx, val_idx in kf.split(X):
            X_train, X_val = X[tr_idx], X[val_idx]
            y_train, y_val = y[tr_idx], y[val_idx]

            model.fit(X_train, y_train)
            pred = model.predict(X_val)
            mape_list.append(mean_absolute_percentage_error(y_val, pred) * 100)

        return float(np.mean(mape_list))

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=50)

    # Save optimization log
    df_trials = pd.DataFrame([
        {**t.params, "MAPE_CV": t.value} for t in study.trials
    ])
    df_trials.to_excel(f"OptimizationLogs/OptimizationLog_YS_{model_name}.xlsx", index=False)

    # Re-evaluate tuned model with 5-fold CV
    best_params = study.best_params
    cv_r2, cv_mape, cv_rmse = [], [], []

    for tr_idx, val_idx in kf.split(X):
        X_train, X_val = X[tr_idx], X[val_idx]
        y_train, y_val = y[tr_idx], y[val_idx]

        tuned_model = build_model(model_class, model_name, best_params)
        tuned_model.fit(X_train, y_train)
        pred = tuned_model.predict(X_val)

        cv_r2.append(r2_score(y_val, pred))
        cv_mape.append(mean_absolute_percentage_error(y_val, pred) * 100)
        cv_rmse.append(np.sqrt(mean_squared_error(y_val, pred)))

    results.append({
        "Model": model_name,
        "R2_mean": np.mean(cv_r2),
        "MAPE_mean": np.mean(cv_mape),
        "RMSE_mean": np.mean(cv_rmse),
        "R2_std": np.std(cv_r2),
        "MAPE_std": np.std(cv_mape),
        "RMSE_std": np.std(cv_rmse)
    })

    final_model = build_model(model_class, model_name, best_params)
    final_model.fit(X, y)
    joblib.dump(final_model, f"YS_best_model_{model_name}.pkl")

# Linear regression baseline
lr = LinearRegression()
cv_r2, cv_mape, cv_rmse = [], [], []

for tr_idx, val_idx in kf.split(X):
    X_train, X_val = X[tr_idx], X[val_idx]
    y_train, y_val = y[tr_idx], y[val_idx]

    lr.fit(X_train, y_train)
    pred = lr.predict(X_val)

    cv_r2.append(r2_score(y_val, pred))
    cv_mape.append(mean_absolute_percentage_error(y_val, pred) * 100)
    cv_rmse.append(np.sqrt(mean_squared_error(y_val, pred)))

results.append({
    "Model": "LR",
    "R2_mean": np.mean(cv_r2),
    "MAPE_mean": np.mean(cv_mape),
    "RMSE_mean": np.mean(cv_rmse),
    "R2_std": np.std(cv_r2),
    "MAPE_std": np.std(cv_mape),
    "RMSE_std": np.std(cv_rmse) 
})

lr.fit(X, y)
joblib.dump(lr, "YS_best_model_LR.pkl")

results_df = pd.DataFrame(results)
results_df.to_excel("All_Models_result_HEA_5fold.xlsx", index=False)